In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor
from itertools import product

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [30]:
my_dates = pd.date_range(start = "2024-03-10", end = date.today()).to_pydatetime().tolist()
my_ticket_types = ["Full","Premier"]
my_arena_levels = ["Clubs", "Uppers", "Lowers", "Premium"]

df = pd.DataFrame(list(product(my_dates, my_ticket_types, my_arena_levels)), columns = ["sale_date", "ticket_type", "arena_level_internal"])

df['day_in_sales_cycle'] =[abs((datetime(2024, 3, 10) - x).days) for x in df['sale_date']]

In [29]:
q = """
SELECT
    greatest(datediff(day, '2024-03-10', date(transaction_date)), 0) AS "day_in_sales_cycle",
    ticket_type,
    arena_level_internal,
    count(*) AS "num_seats_2425",
    sum(gross_revenue) AS "gross_revenue_2425"
FROM
    custom.cth_v_ticket_subscription_2425
WHERE
    is_renewal = FALSE
GROUP BY
    "day_in_sales_cycle", ticket_type, arena_level_internal
ORDER BY
    "day_in_sales_cycle", ticket_type, arena_level_internal"""

df2 = FLA_Redshift(**rs_creds).query_warehouse(sql_string=q)

In [31]:
df3 = pd.merge(left = df, right = df2, how = 'left', on = ["day_in_sales_cycle", "ticket_type", "arena_level_internal"])
df3 = df3.fillna(0)

In [28]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df3, table_name= "cth_daily_bowl_tracker_2425")